In [1]:
from bs4 import BeautifulSoup
import os 
dir_path = os.path.abspath('')


In [2]:
#open the file

#returns a list of tuples formatted as [article title, publish date]
def scrape(filename):
    with open(dir_path+'\\' + filename) as html_file:
        content = html_file.read()

        soup = BeautifulSoup(content, 'lxml')

        #get the table with the articles, and get rid of whitespace
        tabs = soup.find("mw-tabs")

        #each article is in a div with the class being article__content
        articles = tabs.find_all('div', class_="article__content")

        accum = []
        for i in articles:

            #get the title and clean the string of whitespace
            title = i.find('a').stripped_strings

            #making a title and date tuple
            title_date = []

            #I hope there is only one title per article
            for string in title:
                title_date.append(string)

            #get the date from the timestamp span
            title_date.append(i.find('span',class_="article__timestamp")["data-est"])
            accum.append(title_date)
        return accum
                

In [3]:
tuples = scrape(filename='Search - MarketWatch.html')
print(tuples)

[['Oil futures hit five-month high on strong manufacturing data, escalating geopolitical tensions', '2024-04-01T16:07:00'], ['Companies will soon have to disclose how they’re managing climate-related risk. That’s good news for stock investors.', '2024-03-30T12:49:00'], ['The EPA Wants Greener Trucks. Why That’s Fine for Diesel Engine Giant Cummins.', '2024-03-29T13:56:00'], ["TotalEnergies releases its Universal Registration Document 2023 (Document d'enregistrement universel 2023) and its Form 20-F 2023 as well as the proposed resolutions for the Combined Shareholders' Meeting of May 24, 2024", '2024-03-29T12:14:00'], ['ETF flows in first quarter reflect investor hopes for ‘soft landing’', '2024-03-29T04:34:00'], ['TotalEnergies Celebrates its 100(th) Anniversary and Launches the Operation "100 for 100"', '2024-03-29T04:20:00'], ['Mitsui & Co. Plans to Invest in Vietnam Gas Project', '2024-03-29T02:03:00'], ['Oil ends higher, with U.S. prices up 16% for the quarter', '2024-03-28T15:38:

In [4]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification


C:\Users\maxfa\AppData\Local\Temp\ipykernel_24792\1229193036.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\ProgramData\miniconda3\envs\cse578\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [5]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [6]:
#defining the labels to make interpretation easier.
labels = {0:'neutral', 1:'positive',2:'negative'}

In [13]:
#tokenize the first entry as a test
input = tokenizer(tuples[0][0], return_tensors="pt", padding=True)

#enter the tokenized string into finbert and retrieve the output
output = finbert(**input)[0]

#get the actual prediction from the resulting array. This will be the argmax of the output array.
val = labels[np.argmax(output.detach().numpy())]
print(val)
print(output.detach().numpy())

positive
[[-6.342599  10.591372  -5.9413424]]


In [14]:
#this is a test that has a clear negative sentiment. It is the opposite of the first test
test = tokenizer("Oil futures hit five-month Low on weak manufacturing data, escalating geopolitical tensions", return_tensors="pt", padding=True)
test_out = finbert(**test)[0]
val = labels[np.argmax(test_out.detach().numpy())]
print(val)
print(test_out.detach().numpy())

negative
[[-4.665029  -3.9620922 12.77002  ]]
